In [29]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [37]:
training_data_gen = ImageDataGenerator(rotation_range=0.2,shear_range=0.2,
zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,rescale=1./255,
validation_split = 0.2) #created 5 new data points or images from an existing image.
#rescaling because the scale of the original image is way too large.

In [38]:
batchsize = 8

In [39]:
training_data = training_data_gen.flow_from_directory(r'PreparingData\Train',
target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset = 'training')

validation_data = training_data_gen.flow_from_directory(r'PreparingData\Train',
target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset = 'validation')
#we need to give a particular size to the image then only our model will work.

Found 64244 images belonging to 2 classes.
Found 16060 images belonging to 2 classes.


In [40]:
test_data_gen = ImageDataGenerator(rotation_range=0.2)

test_data = test_data_gen.flow_from_directory(r'PreparingData\Test',
target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4594 images belonging to 2 classes.


In [41]:
#Making the base of the model
model_base = InceptionV3(include_top= False,weights='imagenet',
input_tensor=Input(shape=(80,80,3)))
#3 due to lightning conditions

In [42]:
my_model = model_base.output


In [43]:
model_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_282 (Conv2D)            (None, 39, 39, 32)   864         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_282 (Batch  (None, 39, 39, 32)  96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_282 (Activation)    (None, 39, 39, 32)   0           ['batch_normalization_

In [44]:
my_model = Flatten()(my_model)
#deep learning starts from here
my_model = Dense(64,activation='relu')(my_model)
my_model = Dropout(0.5)(my_model)
my_model = Dense(2,activation='softmax')(my_model)
#since this is a categorical model so activation will be softmax.
 #just put those 2 neurons(eye_state)
#drop out to prevent overfitting
#activation= relu means take only important data you can say.

In [45]:
#final model
model = Model(inputs=model_base.input, outputs =my_model)
#since we donot want to train the model from scratch so
for layer in model_base.layers:
    layer.trainable = False

In [46]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_282 (Conv2D)            (None, 39, 39, 32)   864         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_282 (Batch  (None, 39, 39, 32)  96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_282 (Activation)    (None, 39, 39, 32)   0           ['batch_normalization_282[0

In [47]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint = ModelCheckpoint(r'Model\model.h5',monitor='val_loss',save_best_only=True,verbose=3)
#save the best model only based on validation loss
earlystop = EarlyStopping(patience=7,monitor='val_loss',verbose=3,restore_best_weights=True)

rateofLearning = ReduceLROnPlateau(patience=7,verbose=3,monitor='val_loss',)

In [48]:
callbacks = [checkpoint,earlystop,rateofLearning]

In [49]:
#compiling the model
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(training_data,steps_per_epoch=training_data.samples//batchsize,
validation_data = validation_data,
validation_steps = validation_data.samples//batchsize,
callbacks=callbacks,
epochs = 5)

C:\Users\HP\AppData\Local\Temp/ipykernel_13376/239708945.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(training_data,steps_per_epoch=training_data.samples//batchsize,


Epoch 1/5
8030/8030 [==============================] - ETA: 0s - loss: 0.2095 - accuracy: 0.9181
Epoch 00001: val_loss improved from inf to 0.20574, saving model to Model\model.h5
8030/8030 [==============================] - 3283s 407ms/step - loss: 0.2095 - accuracy: 0.9181 - val_loss: 0.2057 - val_accuracy: 0.9121 - lr: 0.0010
Epoch 2/5
1422/8030 [====>.........................] - ETA: 28:33 - loss: 0.1822 - accuracy: 0.9301

KeyboardInterrupt: 

In [16]:
#Model Evaluation for training
training_loss, training_accuracy = model.evaluate_generator(training_data)
print(training_loss)
print(training_accuracy)

C:\Users\HP\AppData\Local\Temp/ipykernel_13376/56489513.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  training_loss, training_accuracy = model.evaluate_generator(training_data)


0.1488361358642578
0.9448197484016418


In [17]:
#Model Evaluation for validation
validation_loss, validation_accuracy = model.evaluate(validation_data)
print(validation_loss)
print(validation_accuracy)

4015/4015 [==============================] - 600s 149ms/step - loss: 0.1924 - accuracy: 0.9249
0.19236500561237335
0.9249066114425659


In [18]:
#Model Evaluation for test
test_loss, test_accuracy = model.evaluate(test_data)
print(test_loss)
print(test_accuracy)

1149/1149 [==============================] - 155s 134ms/step - loss: 102.7964 - accuracy: 0.4867
102.79637145996094
0.48672181367874146
